In [ ]:
import os
import sys
import json
import requests
from typing import Tuple, Optional
from keepa.interface import keepa_minutes_to_time, parse_csv

# Project imports
sys.path.append(os.getcwd())
from src.py.utils.generic_utils import wrapper

In [ ]:
# API Key
api_key = os.environ.get("KEEPA_API_KEY")
api_url = "https://api.keepa.com"

In [ ]:
def get_token_status() -> Tuple[dict, None] | Tuple[None, Exception]:
	'''
		Retrieves token status from Keepa API

		More info: https://keepa.com/#!discuss/t/retrieve-token-status/1305
	'''
	response, err = wrapper(requests.get, f"{api_url}/token", params={"key": api_key})
	if err:
		return None, err
	assert response is not None
	if response.status_code == 200:
		return response.json(), None
	return None, Exception(f"Error: {response.status_code}")

token_status, err = get_token_status()
if err:
	print(err)
else:
	print(f"Token status: {json.dumps(token_status, indent=2)}")

In [ ]:
def get_products_reponse(asin: str, domain_id: int, stats: Optional[Tuple[str, str]] = ("2011-01-01", "2025-01-01"), history: bool = True, rating: bool = True) -> Tuple[dict, None] | Tuple[None, Exception]:
	'''
		Retrieves product information from Keepa API

		More info: https://keepa.com/#!discuss/t/request-products/110

		Args:
			asin: Amazon Standard Identification Number
			domain_id: Amazon domain ID - Valid values: [ 1: com | 2: co.uk 231 | 3: de | 4: fr | 5: co.jp | 6: ca | 8: it | 9: es | 10: in | 11: com.mx ]
			stats: Include product statistics
			history: Include product price history
			rating: Include product rating history
		Returns:
			Product information
	'''
	params = {
		"key": api_key,
		"domain": domain_id,
		"asin": asin,
		"history": 1 if history is True else 0,
		"rating": 1 if rating is True else 0
	}
	if stats:
		params["stats"] = f"{stats[0]},{stats[1]}"
	response, err = wrapper(requests.get, f"{api_url}/product", params=params)
	if err:
		return None, err
	assert response is not None
	if response.status_code == 200:
		response_json = response.json()
		return response_json, None
	return None, Exception(f"Error: {response.status_code}")


In [ ]:
product_response, err = get_products_reponse("B07B428M7F", 1, history=True, rating=True)
if err:
	print(err)
else:
	assert product_response is not None
	print(f"Got {len(product_response['products'])} products")
assert product_response is not None
product = product_response["products"][0]
print(f"Product ASIN: {product['asin']}")
print(f"Product Title: {product['title']}")

In [ ]:
# Product with no stats, history or rating
product_response_1, err = get_products_reponse("B07B428M7F", 1, stats=None, history=False, rating=False)
if err:
	print(err)
product_response_2, err = get_products_reponse("B07B428M7F", 1, stats=None, history=True, rating=False)
if err:
	print(err)
product_response_3, err = get_products_reponse("B07B428M7F", 1, history=True, rating=False)
if err:
	print(err)
product_response_4, err = get_products_reponse("B07B428M7F", 1, history=True, rating=True)



In [ ]:
# Get keys from product response
assert product_response_1 is not None
assert product_response_2 is not None
assert product_response_3 is not None
assert product_response_4 is not None
keys_1 = list(product_response_1["products"][0].keys())
keys_2 = list(product_response_2["products"][0].keys())
keys_3 = list(product_response_3["products"][0].keys())
keys_4 = list(product_response_4["products"][0].keys())
keys_diff_2_1 = set(keys_2) - set(keys_1)
keys_diff_3_1 = set(keys_3) - set(keys_1)
keys_diff_4_1 = set(keys_4) - set(keys_1)

# Print which keys are in 2 but not in 1
print(f"Keys in 2 but not in 1: {keys_diff_2_1}")
print(f"Keys in 3 but not in 1: {keys_diff_3_1}")
print(f"Keys in 4 but not in 1: {keys_diff_4_1}")

# NOTE: All response objects have the same keys, however "csv" field is None if history is False,
#       

# parsed_csv_1 = parse_csv(product_response_1["products"][0]["csv"]) # can't parse None
parsed_csv_2 = parse_csv(product_response_2["products"][0]["csv"])
parsed_csv_3 = parse_csv(product_response_3["products"][0]["csv"])
parsed_csv_4 = parse_csv(product_response_4["products"][0]["csv"])

keys_2 = list(parsed_csv_2.keys())
keys_3 = list(parsed_csv_3.keys())
keys_4 = list(parsed_csv_4.keys())

keys_diff_3_2 = set(keys_3) - set(keys_2)
keys_diff_4_2 = set(keys_4) - set(keys_2)
keys_diff_4_3 = set(keys_4) - set(keys_3)

print(f"Keys in 3 but not in 2: {json.dumps(list(keys_diff_3_2), indent=2)}")
print(f"Keys in 4 but not in 2: {json.dumps(list(keys_diff_4_2), indent=2)}")
print(f"Keys in 4 but not in 3: {json.dumps(list(keys_diff_4_3), indent=2)}")


In [ ]:
# Load a sample downloaded JSON file
sample_product_response_path = "data/keepa/products/domains/1/B0B7CPSN2K.json"
sample_product_response = {}
with open(sample_product_response_path, "r") as f:
	sample_product_response = json.load(f)
sample_parsed_csv = parse_csv(sample_product_response["products"][0]["csv"])
sample_keys = list(sample_parsed_csv.keys())
print(f"Sample keys: {json.dumps(sample_keys, indent=2)}")